In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ISLP import load_data
from statsmodels.stats.multitest import multipletests
import statsmodels.api as sm
pd.set_option('display.float_format', '{:.10f}'.format)


In [61]:
df = load_data('Carseats')
print(df.head())
print(df.dtypes)

          Sales  CompPrice  Income  Advertising  Population  Price ShelveLoc  \
0  9.5000000000        138      73           11         276    120       Bad   
1 11.2200000000        111      48           16         260     83      Good   
2 10.0600000000        113      35           10         269     80    Medium   
3  7.4000000000        117     100            4         466     97    Medium   
4  4.1500000000        141      64            3         340    128       Bad   

   Age  Education Urban   US  
0   42         17   Yes  Yes  
1   65         10   Yes  Yes  
2   59         12   Yes  Yes  
3   55         14   Yes  Yes  
4   38         13   Yes   No  
Sales           float64
CompPrice         int64
Income            int64
Advertising       int64
Population        int64
Price             int64
ShelveLoc      category
Age               int64
Education         int64
Urban          category
US             category
dtype: object


## (a) 
For each quantitative variable in the dataset besides Sales, fit  a linear model to predict Sales using that quantitative variable. Report the p-values associated with the coefficients for the variables. That is, for each model of the form Y= β0 + β1X + ϵ, report the p-value associated with the coefficient β1. Here, Yrepresents Sales and X represents one of the other quantitative variables.

In [131]:
quan_var = df.drop(['Sales','ShelveLoc', 'Urban', 'US'], axis = 1)
y = df['Sales']

    
def linear_model(df, predictor, variable):
    y = predictor
    X = sm.add_constant(df[variable])
    lm = sm.OLS(y,X).fit()
    return lm
    
variables = quan_var.columns
summary_table = pd.DataFrame()
for variable in variables:
    lm = linear_model(quan_var,y, variable)
    output = pd.DataFrame({'Variable' : [variable],
             'p-values' : [lm.pvalues.iloc[1]]})
    summary_table = pd.concat([summary_table,output], ignore_index=True)

print(summary_table)

      Variable     p-values
0    CompPrice 0.2009398289
1       Income 0.0023096705
2  Advertising 0.0000000438
3   Population 0.3139816093
4        Price 0.0000000000
5          Age 0.0000027889
6    Education 0.2999441527


## (b) 
Suppose we control the Type I error at level α = 0.05 for the
p-values obtained in (a). Which null hypotheses do we reject?

In [132]:
H0_rejected = summary_table['p-values'] < 0.05

print('The following variables have a significant effect on Sales at alpha level 0.05:',summary_table[H0_rejected].iloc[:,0])

The following variables have a significant effect on Sales at alpha level 0.05: 1         Income
2    Advertising
4          Price
5            Age
Name: Variable, dtype: object


## (c) 
Now suppose we control the FWER at level 0.05 for the p-values.
Which null hypotheses do we reject?

In [133]:
# correct for multiple testing using bonferroni
p_adjusted = multipletests(summary_table['p-values'],alpha = 0.05, method='bonferroni')
summary_table['bon_corrected'] = p_adjusted[0]  
summary_table['adjusted_pvalue'] = p_adjusted[1]

summary_table

,Variable,p-values,bon_corrected,adjusted_pvalue
0,CompPrice,0.2009398289,False,1.0000000000
1,Income,0.0023096705,True,0.0161676932
2,Advertising,0.0000000438,True,0.0000003064
3,Population,0.3139816093,False,1.0000000000
4,Price,0.0000000000,True,0.0000000000
5,Age,0.0000027889,True,0.0000195226
6,Education,0.2999441527,False,1.0000000000


## (d)
Finally, suppose we control the FDR at level 0.2 for the p-values.
Which null hypotheses do we reject?

In [134]:
p_fdr_adjusted = multipletests(summary_table['p-values'],alpha = 0.05, method='fdr_bh')
summary_table['fdr_corrected'] = p_fdr_adjusted[0]  
summary_table['adjusted_fdr_pvalue'] = p_fdr_adjusted[1]

summary_table

,Variable,p-values,bon_corrected,adjusted_pvalue,fdr_corrected,adjusted_fdr_pvalue
0,CompPrice,0.2009398289,False,1.0000000000,False,0.2813157605
1,Income,0.0023096705,True,0.0161676932,True,0.0040419233
2,Advertising,0.0000000438,True,0.0000003064,True,0.0000001532
3,Population,0.3139816093,False,1.0000000000,False,0.3139816093
4,Price,0.0000000000,True,0.0000000000,True,0.0000000000
5,Age,0.0000027889,True,0.0000195226,True,0.0000065075
6,Education,0.2999441527,False,1.0000000000,False,0.3139816093
